# McNemar Test

## Import All Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.contingency_tables import mcnemar 
import os

## Read All the Data

In [2]:
dir = '/Users/angelaoryza/Documents/TA/noisy-rnnids/rnnids-py/results/vector/mcnemar/smtp'
subdir = os.listdir(dir)

In [3]:
df_dict = {}
df_dict['bi-lstm'] = {}
df_dict['lstm'] = {}
for file in subdir:
    if file.endswith('.csv'):
        df = pd.read_csv(f'./{file}')
        file = file.split('.csv')[0]
        if "bi-lstm" in file:
            file = file.replace('bi-lstm_', '')
            df_dict['bi-lstm'][file] = df
        else:
            file = file.replace('lstm_', '')
            df_dict['lstm'][file] = df

In [4]:
df_dict['bi-lstm']['pred_0.0']

,Unnamed: 0,id,b_mean,b_iqr,b_zscore,f_zscore,f_mean,f_iqr
0,0,175.45.176.0-40755-149.171.126.18-25-tcp,0,1,1,1,0,1
1,1,175.45.176.0-60595-149.171.126.17-25-tcp,0,1,1,1,0,1
2,2,175.45.176.2-58472-149.171.126.17-25-tcp,0,1,1,1,0,1
3,3,175.45.176.2-13352-149.171.126.13-25-tcp,0,1,1,1,0,1
4,4,175.45.176.0-3844-149.171.126.12-25-tcp,0,1,1,1,0,1
...,...,...,...,...,...,...,...,...
44743,44743,59.166.0.9-21800-149.171.126.0-25-tcp,0,0,0,0,0,0
44744,44744,59.166.0.4-6591-149.171.126.6-25-tcp,0,0,0,0,0,0
44745,44745,59.166.0.5-7481-149.171.126.2-25-tcp,0,0,0,0,0,0
44746,44746,59.166.0.0-12790-149.171.126.2-25-tcp,0,0,0,0,0,0


In [5]:
for key, df in df_dict.items():
    for k, val in df_dict[key].items():
        val.drop('Unnamed: 0', axis=1, inplace=True)

## Data Exploration

In [6]:
for key, df in df_dict.items():
    for k, val in df_dict[key].items():
        temp = val[val.duplicated()]['id'].unique()
        print(f'There are {len(temp)} duplicates data in df {k}')

There are 214 duplicates data in df pred_0.0
There are 203 duplicates data in df pred_0.005
There are 201 duplicates data in df pred_0.004
There are 182 duplicates data in df pred_0.006
There are 202 duplicates data in df pred_0.007
There are 193 duplicates data in df pred_0.003
There are 201 duplicates data in df pred_0.002
There are 209 duplicates data in df pred_0.001
There are 202 duplicates data in df pred_0.002
There are 203 duplicates data in df pred_0.003
There are 204 duplicates data in df pred_0.001
There are 214 duplicates data in df pred_0.0
There are 197 duplicates data in df pred_0.004
There are 181 duplicates data in df pred_0.005
There are 212 duplicates data in df pred_0.007
There are 194 duplicates data in df pred_0.006


In [7]:
for key, df in df_dict.items():
    for k, val in df_dict[key].items():
        count = 0
        for t in temp:
            srs = val[val['id']==t][['b_mean', 'b_iqr', 'b_zscore', 'f_mean', 'f_iqr', 'f_zscore' ]].nunique()
            if (srs > 1).any():
                print(t)
                count += 1
                break
            else:
                continue
        if count == 0:
            print(f'All duplicated values in df {key}-{k} have the same prediction')
        else:
            print(f'Not all duplicated values in df {key}-{k} have the same prediction')

149.171.126.17-25-175.45.176.0-51774-tcp
Not all duplicated values in df bi-lstm-pred_0.0 have the same prediction
149.171.126.17-25-175.45.176.0-51774-tcp
Not all duplicated values in df bi-lstm-pred_0.005 have the same prediction
175.45.176.3-18874-149.171.126.17-25-tcp
Not all duplicated values in df bi-lstm-pred_0.004 have the same prediction
175.45.176.0-1249-149.171.126.14-25-tcp
Not all duplicated values in df bi-lstm-pred_0.006 have the same prediction
149.171.126.17-25-175.45.176.0-51774-tcp
Not all duplicated values in df bi-lstm-pred_0.007 have the same prediction
175.45.176.3-18874-149.171.126.17-25-tcp
Not all duplicated values in df bi-lstm-pred_0.003 have the same prediction
149.171.126.17-25-175.45.176.0-51774-tcp
Not all duplicated values in df bi-lstm-pred_0.002 have the same prediction
149.171.126.17-25-175.45.176.0-51774-tcp
Not all duplicated values in df bi-lstm-pred_0.001 have the same prediction
175.45.176.3-18874-149.171.126.17-25-tcp
Not all duplicated values 

## Data Pre-Processing

In [8]:
def preprocess(df):
    df = df.sort_values(by='id')
    df.drop_duplicates(inplace=True)
    return df

In [9]:
for key, df in df_dict.items():
    for k, val in df_dict[key].items():
        val = preprocess(val)

### Merge DF

In [10]:
df_merge = dict()
for key, df in df_dict['lstm'].items():
    x = df_dict['bi-lstm'][key]
    df_all = pd.merge(x, df, on='id', how='inner')
    df_all = df_all.sort_values(by='id')
    df_merge[key] = df_all
        

- Plot F1-Score untuk tiap tiap threshold
- Cari trend regresinya, Hitung gradiennya untuk tiap threshold.
- Dilakukan untuk setiap protokol
- Rekap McNemar

- Vectorizing:


## Run McNemar Test

In [11]:
def mcnemar_test(df, x , y, key):
    a = len(df[(df[x]==0) & (df[y]==0)])
    b = len(df[(df[x]==0) & (df[y]==1)])
    c = len(df[(df[x]==1) & (df[y]==0)])
    d = len(df[(df[x]==1) & (df[y]==1)])

    table = np.array([
    [a,b],
    [c,d]
    ])

    print(key)
    print(table)
    result = mcnemar(table)
    print(result)
    return result.pvalue


### For Binary Score

#### For B-Mean

In [12]:
result = {}
result['b-mean'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_mean_x', 'b_mean_y', key)
    result['b-mean'][key] = test

pred_0.002
[[44561   135]
 [   55   867]]
pvalue      5.998182283273909e-09
statistic   55.0
pred_0.003
[[44520    72]
 [   97   929]]
pvalue      0.06455076034177584
statistic   72.0
pred_0.001
[[44725    56]
 [  132   705]]
pvalue      2.9487792919982617e-08
statistic   56.0
pred_0.0
[[45445    52]
 [   46    75]]
pvalue      0.6137344676301393
statistic   46.0
pred_0.004
[[40336  4331]
 [   67   884]]
pvalue      0.0
statistic   67.0
pred_0.005
[[40363  4181]
 [   50  1024]]
pvalue      0.0
statistic   50.0
pred_0.007
[[40375  4231]
 [   36   976]]
pvalue      0.0
statistic   36.0
pred_0.006
[[44481    89]
 [  108   940]]
pvalue      0.1995641533669113
statistic   89.0


#### For B-IQR

In [13]:
result['b-iqr'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_iqr_x', 'b_iqr_y', key)
    result['b-iqr'][key] = test

pred_0.002
[[40204    90]
 [  151  5173]]
pvalue      0.00010275349920276156
statistic   90.0
pred_0.003
[[39789    94]
 [  404  5331]]
pvalue      8.506785141742995e-47
statistic   94.0
pred_0.001
[[39928   161]
 [  342  5187]]
pvalue      4.886484581305359e-16
statistic   161.0
pred_0.0
[[39752    68]
 [  513  5285]]
pvalue      1.822750230985406e-85
statistic   68.0
pred_0.004
[[39668   408]
 [  319  5223]]
pvalue      0.0010855197503596376
statistic   319.0
pred_0.005
[[39248  1108]
 [   60  5202]]
pvalue      1.509795695839216e-250
statistic   60.0
pred_0.007
[[39532   573]
 [  287  5226]]
pvalue      1.018627072745605e-22
statistic   287.0
pred_0.006
[[39629   521]
 [  301  5167]]
pvalue      1.540897455784566e-14
statistic   301.0


#### For B-ZScore

In [14]:
result['b-zscore'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_zscore_x', 'b_zscore_y', key)
    result['b-zscore'][key] = test

pred_0.002
[[40237    97]
 [  136  5148]]
pvalue      0.01262653982024955
statistic   97.0
pred_0.003
[[40164   116]
 [  238  5100]]
pvalue      8.307901745973495e-11
statistic   116.0
pred_0.001
[[40205   149]
 [  142  5122]]
pvalue      0.7251101640183891
statistic   142.0
pred_0.0
[[40177    74]
 [   96  5271]]
pvalue      0.10698657283154894
statistic   74.0
pred_0.004
[[40139   225]
 [   74  5180]]
pvalue      7.120736815415637e-19
statistic   74.0
pred_0.005
[[39875   509]
 [   62  5172]]
pvalue      2.4530658072404988e-88
statistic   62.0
pred_0.007
[[40139   252]
 [   72  5155]]
pvalue      1.4912458644066358e-24
statistic   72.0
pred_0.006
[[40263   116]
 [  115  5124]]
pvalue      1.0
statistic   115.0


### For Floating Score

#### For F-Mean

In [15]:
result['f-mean'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_mean_x', 'f_mean_y', key)
    result['f-mean'][key] = test

pred_0.002
[[44699    47]
 [  141   731]]
pvalue      4.15705722070781e-12
statistic   47.0
pred_0.003
[[44453    76]
 [  695   394]]
pvalue      5.460547713498193e-126
statistic   76.0
pred_0.001
[[44288   140]
 [   76  1114]]
pvalue      1.58691852771817e-05
statistic   76.0
pred_0.0
[[45276   172]
 [   41   129]]
pvalue      2.8037311964574213e-20
statistic   41.0
pred_0.004
[[45017    28]
 [   32   541]]
pvalue      0.6988834276200058
statistic   28.0
pred_0.005
[[43722  1382]
 [   33   481]]
pvalue      0.0
statistic   33.0
pred_0.007
[[40529  4720]
 [    2   367]]
pvalue      0.0
statistic   2.0
pred_0.006
[[44474   138]
 [  487   519]]
pvalue      1.5169666413270729e-46
statistic   138.0


#### For F-IQR

In [16]:
result['f-iqr'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_iqr_x', 'f_iqr_y', key)
    result['f-iqr'][key] = test

pred_0.002
[[40137   551]
 [  443  4487]]
pvalue      0.0006816903669040145
statistic   443.0
pred_0.003
[[39196   168]
 [ 1507  4747]]
pvalue      3.9283087885603084e-269
statistic   168.0
pred_0.001
[[40071   381]
 [  123  5043]]
pvalue      1.0006531769247524e-31
statistic   123.0
pred_0.0
[[40282   182]
 [  344  4810]]
pvalue      1.4887520732860009e-12
statistic   182.0
pred_0.004
[[40045   377]
 [  210  4986]]
pvalue      5.289380427230023e-12
statistic   210.0
pred_0.005
[[39584   844]
 [ 2186  3004]]
pvalue      1.1564264675262746e-135
statistic   844.0
pred_0.007
[[39943   413]
 [  128  5134]]
pvalue      5.736689799828836e-36
statistic   128.0
pred_0.006
[[39738   490]
 [  715  4675]]
pvalue      9.718942655996354e-11
statistic   490.0


#### For F-ZScore

In [17]:
result['f-zscore'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_zscore_x', 'f_zscore_y', key)
    result['f-zscore'][key] = test

pred_0.002
[[40465   659]
 [  233  4261]]
pvalue      8.699285720010836e-48
statistic   233.0
pred_0.003
[[40369   215]
 [  486  4548]]
pvalue      5.1649382954915245e-25
statistic   215.0
pred_0.001
[[40392   214]
 [  149  4863]]
pvalue      0.0007594968381817871
statistic   149.0
pred_0.0
[[40409   185]
 [  283  4741]]
pvalue      6.817059541081604e-06
statistic   185.0
pred_0.004
[[40379   167]
 [  182  4890]]
pvalue      0.4536621676763865
statistic   167.0
pred_0.005
[[40183   317]
 [ 2383  2735]]
pvalue      0.0
statistic   317.0
pred_0.007
[[40189   261]
 [  152  5016]]
pvalue      9.077111402663694e-08
statistic   152.0
pred_0.006
[[40458   328]
 [  363  4469]]
pvalue      0.19582928730718535
statistic   328.0


## Saving Test Result

In [18]:
mcnemar_result = pd.DataFrame.from_dict(result).sort_index(axis=0)
mcnemar_result

,b-mean,b-iqr,b-zscore,f-mean,f-iqr,f-zscore
pred_0.0,6.137345e-01,1.822750e-85,1.069866e-01,2.803731e-20,1.488752e-12,6.817060e-06
pred_0.001,2.948779e-08,4.886485e-16,7.251102e-01,1.586919e-05,1.000653e-31,7.594968e-04
pred_0.002,5.998182e-09,1.027535e-04,1.262654e-02,4.157057e-12,6.816904e-04,8.699286e-48
pred_0.003,6.455076e-02,8.506785e-47,8.307902e-11,5.460548e-126,3.928309e-269,5.164938e-25
pred_0.004,0.000000e+00,1.085520e-03,7.120737e-19,6.988834e-01,5.289380e-12,4.536622e-01
pred_0.005,0.000000e+00,1.509796e-250,2.453066e-88,0.000000e+00,1.156426e-135,0.000000e+00
pred_0.006,1.995642e-01,1.540897e-14,1.000000e+00,1.516967e-46,9.718943e-11,1.958293e-01
pred_0.007,0.000000e+00,1.018627e-22,1.491246e-24,0.000000e+00,5.736690e-36,9.077111e-08


In [19]:
mcnemar_result.to_excel('./smtp-mcnemar.xlsx')